<!--NAVIGATION-->
< [Design Spaces](01.01-Design-Space-Basics.ipynb) | [Contents](Index.ipynb) | [Results From Parametric Analysis](01.03-Results-From-Parametric-Analysis.ipynb) >

# Making a Sensitivity Analysis

In this chapter, we are taking what we learned about design spaces in the previous chapter, and are going to create a sensitivity analysis for an EnergyPlus model. This process is as follows: first, we take a building energy model, second we prepare a design space which will modify parameters of the model, third we prepare a Job to be run on Amazon Web Services.

Users of this library must be authenticated to use the APIClient. This way, communications with AWS services can only be done by authorized personel and costs can be tracked to users and projects. Authentication is done through an api_key that will be assigned to you.

The standard way to authenticate yourself for the duration of your session in a Jupyter Notebook is to add a block of code as follows. The call to `whoami` returns the username if the authentication is successfull.

In [1]:
%load_ext dotenv
%dotenv

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from surrogate_schema import APIClient

# typical block to authenticate to the API
client = APIClient()
client.whoami()  # shows that you are properly authenticated

User(username='sam@carbonsignal.com', email='sam@carbonsignal.com', full_name='Samuel Letellier-Duchesne', disabled=None, valid_until=datetime.datetime(2025, 6, 11, 15, 44, 24))

In [2]:
from archetypal import IDF

idf = IDF("data/seed_file_V960.idf")

ModuleNotFoundError: No module named 'archetypal'

## Making sure the model is compatible with the workflow

There are a couple things to check to make sure that the model you are working with is compatible with the workflow. First of all, the geometry definition of each thermal zone must be in "World Coordinates". Sometimes, models produced by OpenStudio will generate an IDF model with Zones that are defined in "Relative Coordinates". This means that the vertices defining the walls, windows, overhangs, etc. are defined relative to the Zone origin. The Zone itself can then have X Orgin, Y Origin and Z Origin values that place it in 3D space:

In [ ]:
idf.idfobjects["ZONE"][0]

When that is the case, the model needs to be "translated" to "World Coordinates" such that each vertices is relative to the origin (0,0,0). Fortunately, `archetypal` has a function that can do that automatically: the `IDF.to_world()` method.

In [ ]:
idf.to_world()

In [ ]:
idf.view_model()

In [ ]:
from surrogate_schema import BuildingAnalysis

A BuildingAnalysis class holds all the relevant information regarding a sensitivity analyis (as well as a parametric analysis, but that will come next)

In [ ]:
from surrogate_schema import Space
from surrogate_schema.space.design_space import Dimension

In [ ]:
space = Space(
    dimensions=[
        Dimension(name="WINU", categories=[0.14, 0.3, 0.4, 0.5, 0.8]),
        Dimension(name="WWR", categories=[0.1, 0.3, 0.5, 0.7, 0.9]),
    ],
)

In [ ]:
analysis = BuildingAnalysis(
    company="Elementa",
    project="Tutorial",
    local_epw="data/USA_CA_Chino.AP.722899_TMYx.epw",
    local_seed_model="data/seed_file_V960.idf",
    design_space=space,
    baseline={"WINU": 0.4, "WWR": 0.5},
)

analysis

In [ ]:
analysis.create_space().to_df()

In [ ]:
fig, ax = analysis.sampled_space.pretty_bubble_chart()

In [ ]:
analysis.local_epw

In [ ]:
str(analysis.epw_s3_path)

In [ ]:
analysis.seed_model_s3_path

In [ ]:
analysis.seed_model_s3_path = None

In [ ]:
analysis.prepare_job()

In [ ]:
analysis.submit_job()

In [ ]:
analysis.track_status()

In [ ]:
with open("data/analysis.json", "w") as f:
    f.write(analysis.model_dump_json())

<!--NAVIGATION-->
< [Design Spaces](01.01-Design-Space-Basics.ipynb) | [Contents](Index.ipynb) | [Results From Parametric Analysis](01.03-Results-From-Parametric-Analysis.ipynb) >